In [ ]:
# default_exp generateRandomNumber

# generateRandomNumber
> generates a new number for the user if not already set

In [ ]:
#export
import os, logging, sys, requests
import ujson as json
from awsSchema.apigateway import Event,Response
from src.numberTable import NumberTable
from beartype import beartype
from copy import deepcopy

In [ ]:
#export
############### Logger for debugging code ##################
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

In [ ]:
#export
###################### Error Definitions ######################
class HelperError(Exception): pass
class ParseInputError(HelperError): pass
class CheckDatabaseError(HelperError): pass
class CreateTableError(HelperError): pass
class QueryDatabaseError(HelperError): pass

In [ ]:
#hide
os.environ['USERNUMBERTABLE'] = 'user-number-table-sallee-master'

In [ ]:
#export
################ Setting Globals from Env Vars ################
USERNUMBERTABLE = os.environ['USERNUMBERTABLE']

In [ ]:
#export
########## Helper class for main function ##########
EventInput = dict
class H:
    @staticmethod
    @beartype
    def parseInput(event: EventInput) -> str:
        '''
        returns username and password arguments from input
        '''
        logger.info(f"Number table name :: {USERNUMBERTABLE}")
        evtCpy = deepcopy(event)
        logger.info(f'Event :: {evtCpy}')
        body = Event.parseBody(deepcopy(event))
        try:
            username = body['username']
        except KeyError:
            logger.error('username is not in body')
            raise ParseInputError('username is not in body')

        return username
    
    @staticmethod
    @beartype
    def usernameInDatabase(username: str) -> bool:
        try:
            # queryResult = NumberTable.get(username) # first way
            queryResult = next(NumberTable.query(username),False) # second way
            return True if queryResult else False
        except Exception as e:
            logger.error(f'Unable to check whether or not the username is in the database:\n{e}')
            raise CheckDatabaseError(f'Unable to check whether or not the username is in the database:\n{e}')
    
    @staticmethod
    @beartype
    def getNumber(username: str):
        try:
            user = next(NumberTable.query(username), None)
        except Exception as e:
            logger.error(f'Unable to query database\n{e}')
            raise QueryDatabaseError(f'Unable to query database\n{e}')
        try:
            return user.number
        except Exception as e:
            logger.error(f"User has no number:\n{e}")
            raise QueryDatabaseError(f"User has no number:\n{e}")
    
    @staticmethod
    def getNumber1():
        getIntegerUrl = 'https://www.random.org/integers/?num=1&min=0&max=10&col=1&base=10&format=plain&rnd=new'
        r = requests.get(url=URL)
        if not r.status_code < 400: raise Exception(f'generating number error {r.content}')

    
    @staticmethod
    def saveNumberToTable(username: str, number:int):
        try:
            threadItem = NumberTable(username=deepcopy(username), number=json.loads(number))
            threadItem.save()
        except Exception as e:
            logger.error(f'Unable to add user to database\n{e}')
            raise Exception(f'saving number to database error {e}')
            

NameError: name 'beartype' is not defined

In [ ]:
#export
############################## Main Function ###############################

def generateRandomNumber(event, *args):
  try:
    username = H.parseInput(event)
    if H.usernameInDatabase(username):
      userNumber = H.getNumber(username)
      output = {'success': True, 'age' : 'old', 'number' : int(userNumber)}
      return Response.returnSuccess(body=output)
    number = H.getNumber1()
    H.saveNumberToTable(username, number)
    output = {'success' : True, 'age' : 'new', 'number' : json.loads(number)}
    return Response.returnSuccess(body=output)
  except Exception as e:
    return Response.returnErrro(f'{e}')